In [1]:
# def tanh_function(a_s):
#   return (np.power(np.exp(a_s),2)-1)/(np.power(np.exp(a_s),2)+1)
# def tanh_function_derivative(a_s):
#   fx=tanh_function(a_s)
#   return (1-np.power(fx,2))
# def reLU_function(a_s):
#   return np.maximum(0,a_s)
# def reLU_function_derivative(a_s):
#   fx=reLU_function(a_s)
#   for i in range(len(fx)):
#     if(fx[i]>0):
#       fx[i]=1
#     else: fx[i]=0
#   return fx



# name_of_loss_function="cross_entropy"
# name_of_optimization_function="sgd"
# name_of_weight_function="random"
# max_epochs=5
# weight_decay_value=0.0005
# batch_size_value=32
# name_of_weight_function="sigmoid"
# eta=0.00001
# beta=0.5
# beta1=0.9
# beta2=0.999
# no_of_layers=3
# no_of_neurons_at_each_layer=128
# def initializing_w_and_b_function(no_of_layers,name_of_weight_function,no_of_neurons_at_each_layer):
#   if(name_of_weight_function=="random"):
#     initialize_w_and_b(no_of_layers,no_of_neurons_per_layer)
#   if(name_of_weight_function=="xavier"):
#     weight_intialization_Xavier(no_of_layers,no_of_neurons_per_layer)
# def loss_function(h_s,w_s,b_s,images_to_train,labels_to_train,name_of_loss_function):
#   # if(name_of_loss_function="cross_entropy"):
#   #   cross_entropy_loss_function(h_s,w_s,b_s,images_to_train,labels_to_train)
#   # if(name_of_loss_function="mse"):
#   #   mean_squared_error(h_s,w_s,b_s,images_to_train,labels_to_train)
# def activation_function(name_of_activation_function):
#   # if(name_of_activation_function="sigmoid"):
#   #   sigmoid_function()
#   # if(name_of_activation_function="tanh"):
#   #   tanh_function()
#   # if(name_of_activation_function="ReLU"):
  #   reLU_function()
!pip install wandb
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist,mnist
import numpy as np
import wandb
from wandb.keras import WandbCallback
import socket
socket.setdefaulttimeout(30)  # Set timeout to 30 seconds (adjust as needed)
wandb.login()
wandb.init(project='dL_assignment_1')

(images_to_train, labels_to_train), (test_images,test_labels) = fashion_mnist.load_data()
images_to_train=images_to_train/255.0
test_images=test_images/255.0


def initialize_w_and_b_random(no_of_layers,no_neurons_per_layer):
  W_s=[]
  B_s=[]
  W_s.append(np.random.randn(no_neurons_per_layer,784)*0.01)
  B_s.append(np.random.randn(no_neurons_per_layer,1)*0.01)
  for i in range(1,no_of_layers):
    W_s.append(np.random.randn(no_neurons_per_layer,no_neurons_per_layer)*0.01)
    B_s.append(np.random.randn(no_neurons_per_layer,1)*0.01)
  W_s.append(np.random.randn(10,no_neurons_per_layer)*0.01)
  B_s.append(np.random.randn(10,1)*0.01)
  # print(W_s[0].shape,W_s[-1].shape)
  # print(B_s[0].shape,B_s[-1].shape)
  return W_s,B_s


def weight_intialization_Xavier(no_of_layers,no_neurons_per_layer):
  w_s_xavier=[]
  b_s_xavier=[]
  w_s_xavier.append(np.random.randn(no_neurons_per_layer,784)*0.01)
  b_s_xavier.append(np.zeros((no_neurons_per_layer,1)))
  for i in range(1,no_of_layers):
    w_s_xavier.append(np.random.randn(no_neurons_per_layer,no_neurons_per_layer)*0.01)
    b_s_xavier.append(np.zeros((no_neurons_per_layer,1)))
  w_s_xavier.append(np.random.randn(10,no_neurons_per_layer)*0.01)
  b_s_xavier.append(np.zeros((10,1)))
  # print(w_s_xavier[i].shape for i in range(len(w_s_xavier)))
  # print(b_s_xavier[i].shape for i in range(len(b_s_xavier)))
  return w_s_xavier,b_s_xavier



def intitalize_w_and_b(name_of_function,no_of_layers,no_neurons_per_layer):
  if(name_of_function=="random"):
    return initialize_w_and_b_random(no_of_layers,no_neurons_per_layer)
  if(name_of_function=="xavier"):
    return weight_intialization_Xavier(no_of_layers,no_neurons_per_layer)



def find_loss(h_s,labelled_image,name_of_loss_function):
  if(name_of_loss_function=="cross entropy"):
    # return -np.log(h_s[np.argmax(labelled_image)])
    # return -np.sum(labelled_image*np.log(h_s))
    small_constant=1e-15
    h_s=np.clip(h_s,small_constant,1-small_constant)
    return -np.sum(labelled_image*np.log(h_s))
  # if(name_of_loss_function=="mse"):


def sigmoid_function(a_s):
  # return np.exp(a_s)/(1+np.exp(a_s))
  z_clipped=np.clip(a_s,-500,500)
  return 1/(1+np.exp(-z_clipped))

def tanh_function(a_s):
  return (np.power(np.exp(a_s),2)-1)/(np.power(np.exp(a_s),2)+1)

def reLU_function(a_s):
  return np.maximum(0,a_s)



def activation_function(a_s,name_of_activation):
  if(name_of_activation=="sigmoid"):
    return sigmoid_function(a_s)
  if(name_of_activation=="tanh"):
    return tanh_function(a_s)
  if(name_of_activation=="ReLU"):
    return reLU_function(a_s)

def sigmoid_derivative(a_s):
  fx=sigmoid_function(a_s)
  return fx*(1-fx)

def tanh_function_derivative(a_s):
  fx=tanh_function(a_s)
  return (1-np.power(fx,2))

def reLU_function_derivative(a_s):
  fx=reLU_function(a_s)
  for i in range(len(fx)):
    if(fx[i]>0):
      fx[i]=1
    else: fx[i]=0
  return fx



def derivative_function(a_s,name_of_activation):
  if(name_of_activation=="sigmoid"):
    return sigmoid_derivative(a_s)
  if(name_of_activation=="tanh"):
    return tanh_function_derivative(a_s)
  if(name_of_activation=="ReLU"):
    return reLU_function_derivative(a_s)

def soft_max_function(a_s):
  exps = np.exp(a_s - np.max(a_s))
  # print((exps/np.sum(exps)).shape)
  return exps / np.sum(exps)
  # exps=np.exp(a_s)
  # sum_of_a_s=np.sum(exps)
  # return exps/sum_of_a_s


def calculate_train_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers):
  total_loss=0
  total_accuracy=0
  training_limit=(len(labels_to_train))*(1-validation_split)
  # print(training_limit)
  for i in range(int(training_limit)):
    flattened_image=images_to_train[i].flatten().reshape(784,1)
    a_s,h_s=forward_propagation(w_s,b_s,flattened_image,no_of_layers,images_to_train,name_of_activation)
    labelled_image=np.zeros((10,1))
    labelled_image[labels_to_train[i]]=1
    total_loss+=find_loss(h_s[-1],labelled_image,name_of_loss_function)
    if(np.argmax(h_s[-1])==labels_to_train[i]):
      total_accuracy+=1
  return (total_loss/training_limit),(total_accuracy/training_limit)*100



def calculate_valid_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers):
  validation_loss=0
  validation_accuracy=0
  validation_limit=(len(labels_to_train))*(1-validation_split)
  for i in range(int(validation_limit),int(len(labels_to_train))):
    flattened_image=images_to_train[i].flatten().reshape(784,1)
    a_s,h_s=forward_propagation(w_s,b_s,flattened_image,no_of_layers,images_to_train,name_of_activation)
    labelled_image=np.zeros((10,1))
    labelled_image[labels_to_train[i]]=1
    validation_loss+=find_loss(h_s[-1],labelled_image,name_of_loss_function)
    if(np.argmax(h_s[-1])==labels_to_train[i]):
      validation_accuracy+=1
  return (validation_loss/(len(labels_to_train)*validation_split)),(validation_accuracy/(len(labels_to_train)*validation_split))*100



def forward_propagation(w_s,b_s,flattened_image,no_of_layers,images_to_train,name_of_activation):
  a_s=[]
  h_s=[]
  h_s.append(flattened_image)
  for i in range(no_of_layers):
    a_s.append(np.matmul(w_s[i],flattened_image)+b_s[i])
    h_s.append(activation_function(a_s[i],name_of_activation))
    flattened_image=h_s[-1]
  a_s.append(np.dot(w_s[-1],flattened_image)+b_s[-1])
  h_s.append(soft_max_function(a_s[-1]))
  # print(h_s[-1].shape)
  # print(a_s[0].shape,a_s[-1].shape)
  # print(h_s[0].shape,h_s[-1].shape)
  return a_s,h_s


def backward_propagation(a_s,h_s,label_image,w_s,b_s,no_of_layers,name_of_activation):
  theta_w=[]
  theta_b=[]
  nabla_a_last=-(label_image-h_s[-1])
  for k in range(no_of_layers,-1,-1):
    nabla_w_last=np.matmul(nabla_a_last,h_s[k].T)
    theta_w.append(nabla_w_last)
    nabla_b_last=nabla_a_last
    theta_b.append(nabla_b_last)
    if(k==0):
      break
    nabla_h_last=np.matmul(w_s[k].T,nabla_a_last)
    # print(w_s[k].T.shape,nabla_a_last.shape)
    nabla_a_last=np.multiply(nabla_h_last,derivative_function(a_s[k-1],name_of_activation))
  return theta_w,theta_b


  # nabla_a_last=-(label_image-h_s[-1])
  # for k in range(no_of_layers-1,-1,-1):
  #   nabla_w_last=np.matmul(nabla_a_last,h_s[k+1].T)
  #   theta_w.append(nabla_w_last)
  #   nabla_b_last=nabla_a_last
  #   theta_b.append(nabla_b_last)
  #   nabla_h_last=np.matmul(w_s[k].T,nabla_a_last)
  #   # print(w_s[k].T.shape,nabla_a_last.shape)
  #   nabla_a_last=np.multiply(nabla_h_last,derivative_function(a_s[k],name_of_activation))
  # nabla_w_last=np.matmul(nabla_a_last,h_s[0].T)
  # theta_w.append(nabla_w_last)
  # nabla_b_last=nabla_a_last
  # theta_b.append(nabla_b_last)
  # return theta_w,theta_b



def stochastic_gradient_descent(w_s,b_s,labels_to_train,validation_split,eta,max_epochs,name_of_activation,weight_decay,name_of_derivation_function,name_of_loss_function,no_of_layers):
  batch_size=1
  for iterations in range(max_epochs):
    total_accuracy=0
    total_loss=0
    dw=[np.zeros_like(every_ele) for every_ele in w_s]
    db=[np.zeros_like(every_ele) for every_ele in b_s]
    for i in range(int(60000*1-(60000*validation_split))):
      label_image=np.zeros((10,1))
      label_image[labels_to_train[i]]=1
      a_s,h_s=forward_propagation(w_s,b_s,images_to_train[i].flatten().reshape((784,1)),no_of_layers,images_to_train,name_of_activation)
      grad_w,grad_b=backward_propagation(a_s,h_s,label_image,w_s,b_s,no_of_layers,name_of_activation)
      grad_w.reverse()
      grad_b.reverse()
      dw = [dw_current + grad for dw_current, grad in zip(dw, grad_w)]
      db = [db_current + grad for db_current, grad in zip(db, grad_b)]
      if (i + 1) % batch_size == 0:
          w_s = [w - (eta * grad) -(w*weight_decay) for w, grad in zip(w_s, dw)]
          b_s = [b- eta * grad for b, grad in zip(b_s, db)]
    # accuracy=calculate_accuracy_and_loss(images_to_train,labels_to_train,w_s,b_s,h_s)
    # # validation_accuracy,validation_loss=cal_val_acc_and_loss(h_s,images_to_train,validation_split)
    # # loss=(total_loss/(60000*(1-validation_split)))
    # loss=cross_entropy_loss_function(h_s,w_s,b_s,images_to_train,labels_to_train)
    # accuracy=(total_accuracy/(60000*(1-validation_split)))
    # valid_loss=(validation_loss/(60000*(validation_split)))
    # valid_accuracy=(validation_accuracy/(60000*(validation_split)))
    train_loss,train_accuracy=calculate_train_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers)
    valid_loss,valid_accuracy=calculate_valid_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers)
    print(f"training loss: {train_loss},training accuracy: {train_accuracy}, validation accuracy: {valid_accuracy}, validation loss: {valid_loss}")
    wandb.log({"train_loss":train_loss,"train_accuracy":train_accuracy,"valid_loss":valid_loss,"valid_accuracy":valid_accuracy})



def momentum_gradient_descent(w_s,b_s,labels_to_train,validation_split,eta,max_epochs,name_of_activation,beta,mini_batch_size,weight_decay,name_of_derivation_function,name_of_loss_function,no_of_layers):
  last_uw=[np.zeros_like(every_ele) for every_ele in w_s]
  last_ub=[np.zeros_like(every_ele) for every_ele in b_s]
  for iterations in range(max_epochs):
    # total_accuracy=0
    # total_loss=0
    dw=[np.zeros_like(every_ele) for every_ele in w_s]
    db=[np.zeros_like(every_ele) for every_ele in b_s]
    points_touched_till_now=0
    for i in range(int(60000*1-(60000*validation_split))):
      label_image=np.zeros((10,1))
      label_image[labels_to_train[i]]=1
      a_s,h_s=forward_propagation(w_s,b_s,images_to_train[i].flatten().reshape((784,1)),no_of_layers,images_to_train,name_of_activation)
      # total_loss+=find_loss(h_s[-1],label_image,)
      # if(np.argmax(h_s[-1])==labels_to_train[i]):
        # total_accuracy+=1
      grad_w,grad_b=backward_propagation(a_s,h_s,label_image,w_s,b_s,no_of_layers,name_of_activation)
      grad_w.reverse()
      grad_b.reverse()
      dw = [dw_current + grad for dw_current, grad in zip(dw, grad_w)]
      db = [db_current + grad for db_current, grad in zip(db, grad_b)]
      if (i + 1) % mini_batch_size == 0:
          u_w = [beta * uw + grad for uw, grad in zip(last_uw, dw)]
          u_b = [beta * ub + grad for ub, grad in zip(last_ub, db)]
          w_s = [w*(1-weight_decay) - eta * uw for w, uw in zip(w_s, u_w)]
          b_s = [b - eta * ub for b, ub in zip(b_s, u_b)]
          last_uw = u_w
          last_ub = u_b

    # validation_accuracy,validation_loss=cal_val_acc_and_loss(h_s,images_to_train,validation_split)
    # # loss=(total_loss/(60000*(1-validation_split)))
    # cross_entropy_loss_function(h_s,w_s,b_s,images_to_train,labels_to_train)
    # # accuracy=(total_accuracy/(60000*(1-validation_split)))
    # accuracy=calculate_accuracy_and_loss(images_to_train,labels_to_train,w_s,b_s,h_s)
    # valid_loss=(validation_loss/(60000*(validation_split)))
    # valid_accuracy=(validation_accuracy/(60000*(validation_split)))
    train_loss,train_accuracy=calculate_train_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers)
    valid_loss,valid_accuracy=calculate_valid_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers)
    print(f"training loss: {train_loss},training accuracy: {train_accuracy}, validation accuracy: {valid_accuracy}, validation loss: {valid_loss}")
    wandb.log({"train_loss":train_loss,"train_accuracy":train_accuracy,"valid_loss":valid_loss,"valid_accuracy":valid_accuracy})




def nesterov_gradient_descent(w_s,b_s,labels_to_train,validation_split,eta,max_epochs,name_of_activation,beta,mini_batch_size,weight_decay,name_of_derivation_function,name_of_loss_function,no_of_layers):
  last_uw=[np.zeros_like(every_ele) for every_ele in w_s]
  last_ub=[np.zeros_like(every_ele) for every_ele in b_s]
  for iterations in range(max_epochs):
    # total_accuracy=0
    # total_loss=0
    dw=[np.zeros_like(every_ele) for every_ele in w_s]
    db=[np.zeros_like(every_ele) for every_ele in b_s]
    points_touched_till_now=0
    u_w = [beta * uw for uw in last_uw]
    u_b = [beta * ub for ub in last_ub]
    for i in range(int(60000*1-(60000*validation_split))):
      label_image=np.zeros((10,1))
      label_image[labels_to_train[i]]=1
      a_s,h_s=forward_propagation(w_s,b_s,images_to_train[i].flatten().reshape((784,1)),no_of_layers,images_to_train,name_of_activation)
      # total_loss+=find_loss(h_s[-1],label_image)
      # if(np.argmax(h_s[-1])==labels_to_train[i]):
        # total_accuracy+=1
      grad_w,grad_b=backward_propagation(a_s,h_s,label_image,[w-u for w, u in zip(w_s,u_w)],[b-u for b,u in zip(b_s,u_b)],no_of_layers,name_of_activation)
      grad_w.reverse()
      grad_b.reverse()
      dw = [dw_current + grad for dw_current, grad in zip(dw, grad_w)]
      db = [db_current + grad for db_current, grad in zip(db, grad_b)]
      if (i + 1) % mini_batch_size == 0:
        u_w = [beta * uw + grad for uw, grad in zip(last_uw, dw)]
        u_b = [beta * ub + grad for ub, grad in zip(last_ub, db)]
        w_s = [w*(1-weight_decay) - eta * uw for w, uw in zip(w_s, u_w)]
        b_s = [b - eta * ub for b, ub in zip(b_s, u_b)]
        last_uw = u_w
        last_ub = u_b
    # validation_accuracy,validation_loss=cal_val_acc_and_loss(h_s,images_to_train,validation_split)
    # # loss=(total_loss/(60000*(1-validation_split)))
    # cross_entropy_loss_function(h_s,w_s,b_s,images_to_train,labels_to_train)
    # # accuracy=(total_accuracy/(60000*(1-validation_split)))
    # accuracy=calculate_accuracy_and_loss(images_to_train,labels_to_train,w_s,b_s,h_s)
    # valid_loss=(validation_loss/(60000*(validation_split)))
    # valid_accuracy=(validation_accuracy/(60000*(validation_split)))
    train_loss,train_accuracy=calculate_train_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers)
    valid_loss,valid_accuracy=calculate_valid_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers)
    print(f"training loss: {train_loss},training accuracy: {train_accuracy}, validation accuracy: {valid_accuracy}, validation loss: {valid_loss}")
    wandb.log({"train_loss":train_loss,"train_accuracy":train_accuracy,"valid_loss":valid_loss,"valid_accuracy":valid_accuracy})




def rmsprop_gradient_descent(w_s,b_s,labels_to_train,validation_split,eta,max_epochs,name_of_activation,beta,mini_batch_size,eps,weight_decay,name_of_derivation_function,name_of_loss_function,no_of_layers):
  last_uw=[np.zeros_like(every_ele) for every_ele in w_s]
  last_ub=[np.zeros_like(every_ele) for every_ele in b_s]
  for iterations in range(max_epochs):
    # total_accuracy=0
    # total_loss=0
    dw=[np.zeros_like(every_ele) for every_ele in w_s]
    db=[np.zeros_like(every_ele) for every_ele in b_s]
    points_touched_till_now=0
    for i in range(int(60000*1-(60000*validation_split))):
      label_image=np.zeros((10,1))
      label_image[labels_to_train[i]]=1
      a_s,h_s=forward_propagation(w_s,b_s,images_to_train[i].flatten().reshape((784,1)),no_of_layers,images_to_train,name_of_activation)
      # total_loss+=find_loss(h_s[-1],label_image)
      # if(np.argmax(h_s[-1])==labels_to_train[i]):
        # total_accuracy+=1
      grad_w,grad_b=backward_propagation(a_s,h_s,label_image,w_s,b_s,no_of_layers,name_of_activation)
      grad_w.reverse()
      grad_b.reverse()
      dw = [dw_current + grad for dw_current, grad in zip(dw, grad_w)]
      db = [db_current + grad for db_current, grad in zip(db, grad_b)]
      if((i+1)%mini_batch_size==0):
        u_w = [beta * uw + (1-beta)*(grad**2) for uw, grad in zip(last_uw, dw)]
        u_b = [beta * ub + (1-beta)*(grad**2) for ub, grad in zip(last_ub, db)]
        w_s = [w*(1-weight_decay) - eta * x / (np.sqrt(uw + eps)) for w, x, uw in zip(w_s, dw, u_w)]
        b_s = [b - eta * x / (np.sqrt(ub + eps)) for b, x, ub in zip(b_s, db, u_b)]
        last_uw=u_w
        last_ub=u_b
    # validation_accuracy,validation_loss=cal_val_acc_and_loss(h_s,images_to_train,validation_split)
    # # loss=(total_loss/(60000*(1-validation_split)))
    # cross_entropy_loss_function(h_s,w_s,b_s,images_to_train,labels_to_train)
    # # accuracy=(total_accuracy/(60000*(1-validation_split)))
    # accuracy=calculate_accuracy_and_loss(images_to_train,labels_to_train,w_s,b_s,h_s)
    # valid_loss=(validation_loss/(60000*(validation_split)))
    # valid_accuracy=(validation_accuracy/(60000*(validation_split)))
    train_loss,train_accuracy=calculate_train_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers)
    valid_loss,valid_accuracy=calculate_valid_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers)
    print(f"training loss: {train_loss},training accuracy: {train_accuracy}, validation accuracy: {valid_accuracy}, validation loss: {valid_loss}")
    wandb.log({"train_loss":train_loss,"train_accuracy":train_accuracy,"valid_loss":valid_loss,"valid_accuracy":valid_accuracy})



def adam_gradient_descent(w_s,b_s,labels_to_train,validation_split,eta,max_epochs,name_of_activation,beta,beta1,beta2,eps,weight_decay,name_of_derivation_function,name_of_loss_function,no_of_layers,mini_batch_size):
  m_w=[np.zeros_like(every_ele) for every_ele in w_s]
  m_b=[np.zeros_like(every_ele) for every_ele in b_s]
  v_w=[np.zeros_like(every_ele) for every_ele in w_s]
  v_b=[np.zeros_like(every_ele) for every_ele in b_s]
  for iterations in range(max_epochs):
    # total_accuracy=0
    # total_loss=0
    dw=[np.zeros_like(every_ele) for every_ele in w_s]
    db=[np.zeros_like(every_ele) for every_ele in b_s]
    points_touched_till_now=0
    for i in range(int(60000*1-(60000*validation_split))):
      label_image=np.zeros((10,1))
      label_image[labels_to_train[i]]=1
      a_s,h_s=forward_propagation(w_s,b_s,images_to_train[i].flatten().reshape((784,1)),no_of_layers,images_to_train,name_of_activation)
      # total_loss+=find_loss(h_s[-1],label_image)
      # if(np.argmax(h_s[-1])==labels_to_train[i]):
      #   total_accuracy+=1
      grad_w,grad_b=backward_propagation(a_s,h_s,label_image,w_s,b_s,no_of_layers,name_of_activation)
      grad_w.reverse()
      grad_b.reverse()
      dw = [dw_current + grad for dw_current, grad in zip(dw, grad_w)]
      db = [db_current + grad for db_current, grad in zip(db, grad_b)]
      if((i+1)%mini_batch_size==0):
        m_w = [beta1* mw + (1-beta1)*(grad) for mw, grad in zip(m_w, dw)]
        m_b = [beta1* mb + (1-beta1)*(grad) for mb, grad in zip(m_b, db)]
        m_w = [beta2* vw + (1-beta2)*(grad**2) for vw, grad in zip(v_w, dw)]
        m_b = [beta2* vb + (1-beta2)*(grad**2) for vb, grad in zip(v_b, db)]


        m_w_hat=[m/(1-np.power(beta1,i+1)) for m in m_w]
        m_b_hat=[m/(1-np.power(beta1,i+1)) for m in m_b]
        v_w_hat=[v/(1-np.power(beta2,i+1)) for v in v_w]
        v_b_hat=[v/(1-np.power(beta2,i+1)) for v in v_b]
        for i in range(len(w_s)):
          for j in range(len(w_s[i])):
              w_s[i][j] =w_s[i][j]*(1-weight_decay)-(eta * m_w_hat[i][j]) / (np.sqrt(v_w_hat[i][j]) + eps)
              b_s[i][j]-= (eta * m_b_hat[i][j]) / (np.sqrt(v_b_hat[i][j]) + eps)
    # validation_accuracy,validation_loss=cal_val_acc_and_loss(h_s,images_to_train,validation_split)
    # # loss=(total_loss/(60000*(1-validation_split)))
    # cross_entropy_loss_function(h_s,w_s,b_s,images_to_train,labels_to_train)
    # # accuracy=(total_accuracy/(60000*(1-validation_split)))
    # accuracy=calculate_accuracy_and_loss(images_to_train,labels_to_train,w_s,b_s,h_s)
    # valid_loss=(validation_loss/(60000*(validation_split)))
    # valid_accuracy=(validation_accuracy/(60000*(validation_split)))
    train_loss,train_accuracy=calculate_train_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers)
    valid_loss,valid_accuracy=calculate_valid_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers)
    print(f"training loss: {train_loss},training accuracy: {train_accuracy}, validation accuracy: {valid_accuracy}, validation loss: {valid_loss}")
    wandb.log({"train_loss":train_loss,"train_accuracy":train_accuracy,"valid_loss":valid_loss,"valid_accuracy":valid_accuracy})



def nadam_gradient_descent(w_s,b_s,labels_to_train,validation_split,eta,max_epochs,name_of_activation,beta,beta1,beta2,eps,weight_decay,name_of_derivation_function,name_of_loss_function,no_of_layers,mini_batch_size):
  m_w=[np.zeros_like(every_ele) for every_ele in w_s]
  m_b=[np.zeros_like(every_ele) for every_ele in b_s]
  v_w=[np.zeros_like(every_ele) for every_ele in w_s]
  v_b=[np.zeros_like(every_ele) for every_ele in b_s]
  for iterations in range(max_epochs):
    # total_accuracy=0
    # total_loss=0
    dw=[np.zeros_like(every_ele) for every_ele in w_s]
    db=[np.zeros_like(every_ele) for every_ele in b_s]
    # points_touched_till_now=0
    for i in range(int(60000*1-(60000*validation_split))):
      label_image=np.zeros((10,1))
      label_image[labels_to_train[i]]=1
      a_s,h_s=forward_propagation(w_s,b_s,images_to_train[i].flatten().reshape((784,1)),no_of_layers,images_to_train,name_of_activation)
      # total_loss+=find_loss(h_s[-1],label_image)
      # if(np.argmax(h_s[-1])==labels_to_train[i]):
      #   total_accuracy+=1
      grad_w,grad_b=backward_propagation(a_s,h_s,label_image,w_s,b_s,no_of_layers,name_of_activation)
      grad_w.reverse()
      grad_b.reverse()
      dw = [dw_current + grad for dw_current, grad in zip(dw, grad_w)]
      db = [db_current + grad for db_current, grad in zip(db, grad_b)]
      if((i+1)%mini_batch_size==0):
        m_w = [beta1* mw + (1-beta1)*(grad) for mw, grad in zip(m_w, dw)]
        m_b = [beta1* mb + (1-beta1)*(grad) for mb, grad in zip(m_b, db)]
        v_w = [beta2* vw + (1-beta2)*(grad**2) for vw, grad in zip(v_w, dw)]
        v_b = [beta2* vb + (1-beta2)*(grad**2) for vb, grad in zip(v_b, db)]
        m_w_hat=[m/(1-np.power(beta1,i+1)) for m in m_w]
        m_b_hat=[m/(1-np.power(beta1,i+1)) for m in m_b]
        v_w_hat=[v/(1-np.power(beta2,i+1)) for v in v_w]
        v_b_hat=[v/(1-np.power(beta2,i+1)) for v in v_b]
        for i in range(len(w_s)):
          for j in range(len(w_s[i])):
              update_w = np.dot((eta / (np.sqrt(v_w_hat[i][j]) + eps)), (beta1 * m_w_hat[i][j] + (((1 - beta1) * dw[i][j]) / (1 - np.power(beta1, i + 1)))))
              w_s[i][j] =w_s[i][j]*(1-weight_decay)-update_w
        for i in range(len(b_s)):
          for j in range(len(b_s[i])):
              update_b = np.dot((eta / (np.sqrt(v_b_hat[i][j]) + eps)), (beta1 * m_b_hat[i][j] + (((1 - beta1) * db[i][j]) / (1 - np.power(beta1, i + 1)))))
              b_s[i][j] -= update_b
    # validation_accuracy,validation_loss=cal_val_acc_and_loss(h_s,images_to_train,validation_split)
    # # loss=(total_loss/(60000*(1-validation_split)))
    # cross_entropy_loss_function(h_s,w_s,b_s,images_to_train,labels_to_train)
    # # accuracy=(total_accuracy/(60000*(1-validation_split)))
    # accuracy=calculate_accuracy_and_loss(images_to_train,labels_to_train,w_s,b_s,h_s)
    # valid_loss=(validation_loss/(60000*(validation_split)))
    # valid_accuracy=(validation_accuracy/(60000*(validation_split)))
    train_loss,train_accuracy=calculate_train_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers)
    valid_loss,valid_accuracy=calculate_valid_loss_and_accuracy(name_of_loss_function,w_s,b_s,h_s,images_to_train,labels_to_train,validation_split,name_of_activation,no_of_layers)
    print(f"training loss: {train_loss},training accuracy: {train_accuracy}, validation accuracy: {valid_accuracy}, validation loss: {valid_loss}")
    # wandb.log({'valid_loss':valid_loss})
    wandb.log({"train_loss":train_loss,"train_accuracy":train_accuracy,"valid_loss":valid_loss,"valid_accuracy":valid_accuracy})




def optimization_function(name_of_optimizer,w_s,b_s,labels_to_train,validation_split,eta,max_epochs,beta,mini_batch_size,eps,beta1,beta2,weight_decay,name_of_activation,
                        name_of_loss_function,name_of_derivation_function,no_of_layers):
  if(name_of_optimizer=="sgd"):
    stochastic_gradient_descent(w_s,b_s,labels_to_train,validation_split,eta,max_epochs,name_of_activation,weight_decay,name_of_derivation_function,name_of_loss_function,no_of_layers)
  if(name_of_optimizer=="momentum"):
    momentum_gradient_descent(w_s,b_s,labels_to_train,validation_split,eta,max_epochs,name_of_activation,beta,mini_batch_size,weight_decay,name_of_derivation_function,name_of_loss_function,no_of_layers)
  if(name_of_optimizer=="nesterov"):
    nesterov_gradient_descent(w_s,b_s,labels_to_train,validation_split,eta,max_epochs,name_of_activation,beta,mini_batch_size,weight_decay,name_of_derivation_function,name_of_loss_function,no_of_layers)
  if(name_of_optimizer=="rmsprop"):
    rmsprop_gradient_descent(w_s,b_s,labels_to_train,validation_split,eta,max_epochs,name_of_activation,beta,mini_batch_size,eps,weight_decay,name_of_derivation_function,name_of_loss_function,no_of_layers)
  if(name_of_optimizer=="adam"):
    adam_gradient_descent(w_s,b_s,labels_to_train,validation_split,eta,max_epochs,name_of_activation,beta,beta1,beta2,eps,weight_decay,name_of_derivation_function,name_of_loss_function,no_of_layers,mini_batch_size)
  if(name_of_optimizer=="nadam"):
    nadam_gradient_descent(w_s,b_s,labels_to_train,validation_split,eta,max_epochs,name_of_activation,beta,beta1,beta2,eps,weight_decay,name_of_derivation_function,name_of_loss_function,no_of_layers,mini_batch_size)
# def no_of_epochs(max_epochs)
# def weight_decay(weight_decay_value)
# def batch_size(batch_size_value)
# name_of_function="random"
# no_of_layers=4
# no_neurons_per_layer=32
# name_of_optimizer="sgd"
# validation_split=0.1
# eta=0.001
# eps=1e-5
# beta=0.5
# beta1=0.9
# beta2=0.999
# max_epochs=10
# mini_batch_size=64
# weight_decay=0.5
# name_of_activation="ReLU"
# name_of_loss_function="cross entropy"
name_of_derivation_function="sigmoid_derivative"



# def main_function(name_of_function,no_of_layers,no_neurons_per_layer,name_of_optimizer,labels_to_train,validation_split,eta,max_epochs,beta,mini_batch_size,eps,beta1,beta2,name_of_activation,
#                         name_of_loss_function,weight_decay,name_of_derivation_function):
#   w_s,b_s=intitalize_w_and_b(name_of_function,no_of_layers,no_neurons_per_layer)
#   optimization_function(name_of_optimizer,w_s,b_s,labels_to_train,validation_split,eta,max_epochs,beta,mini_batch_size,eps,beta1,beta2,weight_decay,name_of_activation,
#                         name_of_loss_function,name_of_derivation_function)

# !pip install wandb
# import wandb
# wandb.login()
# wandb.init(project="CS6910_dL_assignment_1")

def main_function():
  wandb.init(project='dL_assignment_1')
  params=wandb.config
  with wandb.init(project='dL_assignment_1',name='fun->'+params.name_of_function) as run :
    w_s,b_s=intitalize_w_and_b(params.name_of_function,params.no_of_layers,params.no_neurons_per_layer)
    optimization_function(params.name_of_optimizer,w_s,b_s,labels_to_train,0.1,params.eta,params.max_epochs,0.5,params.mini_batch_size,params.eps,0.9,0.999,params.weight_decay,params.name_of_activation,
                        params.name_of_loss_function,name_of_derivation_function,params.no_of_layers)
# main_function(name_of_function,no_of_layers,no_neurons_per_layer,name_of_optimizer,labels_to_train,validation_split,eta,max_epochs,beta,mini_batch_size,eps,beta1,beta2,name_of_activation,
#               name_of_loss_function,weight_decay,name_of_derivation_function)


sweep_params={
    'method':'bayes',
    'name': 'test',
    'metric':{
        'goal':'maximize',
        'name':'valid_accuracy'
    },
    'parameters':{
        'name_of_function':{'values':['random','xavier']},
        'no_of_layers':{'values':[3,4,5]},
        'no_neurons_per_layer':{'values':[32,16,64]},
        'name_of_optimizer':{'values':['sgd','momentum','nesterov','rmsprop','adam','nadam']},
        'eta':{'values':[1e-3,1e-4]},
        'eps':{'values':[1e-5,1e-8,1e-10]},
        'mini_batch_size':{'values':[16,32,64]},
        'weight_decay':{'values':[0.0005,0,0.5]},
        'max_epochs':{'values':[5,10]},
        'name_of_activation':{'values':['tanh','sigmoid','ReLU']},
        'name_of_loss_function':{'values':['cross entropy']}
    }
}

sid=wandb.sweep(sweep_params,project='dL_assignment_1')
wandb.agent(sid,function=main_function,count=5)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ksharathkumar111 (cs23m033_dl_assignment_1). Use `wandb login --relogin` to force relogin


...

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


training loss: 2.3043124501539753,training accuracy: 9.944444444444445, validation accuracy: 10.5, validation loss: 2.3033778757810226
training loss: 2.3032609013605367,training accuracy: 10.064814814814815, validation accuracy: 9.416666666666666, validation loss: 2.3042295997031195
training loss: 2.303479868411754,training accuracy: 9.944444444444445, validation accuracy: 10.5, validation loss: 2.302922483526487
training loss: 2.3033636579626453,training accuracy: 10.064814814814815, validation accuracy: 9.416666666666666, validation loss: 2.304327663571716
training loss: 2.3037032892795772,training accuracy: 9.944444444444445, validation accuracy: 10.5, validation loss: 2.3029033389256464
training loss: 2.3031416930870927,training accuracy: 9.938888888888888, validation accuracy: 10.549999999999999, validation loss: 2.3034672099045372
training loss: 2.3033654469420832,training accuracy: 9.944444444444445, validation accuracy: 10.5, validation loss: 2.302852993938623
training loss: 2.

train_accuracy,▁▇▁▇▁▁▁█▁█
train_loss,█▃▄▃▅▂▃▁▄▁
valid_accuracy,█▂█▂███▁█▁
valid_loss,▃█▁█▁▄▁▄▁▂
train_accuracy,10.08333
train_loss,2.30295
valid_accuracy,9.25
valid_loss,2.30307


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fc13be63730>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe